In [1]:
# install necessary packages using pip
!pip install keras numpy wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=21e9757e8f306c0e74408822c3fac24b0501b581a67efabf88f63c9e28d8e7e4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import os
import io
import sys
import wget

In [3]:
# NOTE*: The train.zip file contains all 500 (unzipped) documents, 
#   so it only has to be unzipped once.

fileName = "/content/train.zip"

# Checks if the file already exists.
if os.path.isfile(fileName):
    print("'" + fileName + "' already exists.")

# Otherwise, it downloads it.    
else:
    url = "https://raw.githubusercontent.com/rks170530/NLP_Project1_Corpus/main/train.zip"
    wget.download(url, fileName)

In [4]:
# NOTE**: In case the wget.download() does not work for whatever reason, 
#   please use the "train.zip" file included in the Project1 folder, uncomment
#   the next 3 lines and upload the file when prompted.

# from google.colab import files
# files.upload()

In [5]:
from zipfile import ZipFile

# Extract the Zipfile into the 500 documents.
with ZipFile(fileName, 'r') as zip:
  zip.extractall()


In [6]:
def load_corpus(path):

    # Check if the path is a directory.
    if not os.path.isdir(path):
        sys.exit("Input path is not a directory")

    sentenceArray = []

    # For each file in the path.
    for fileName in os.listdir(path):
        fileName = os.path.join(path, fileName)
    
        # Open and read the file.
        try:
            fileReader = io.open(fileName, 'r', encoding='utf-8', errors='ignore')

            # Until there are no more lines.
            while True:
                    
                # Read the line.
                wordArray = []
                currentLine = fileReader.readline()

                # Exit if there are no new lines.
                if not currentLine:
                    break
                
                # Remove end-line and convert to lowercase.
                currentLine = currentLine.replace('\n', "")

                # Save the line only if it isn't empty.
                if (currentLine != ""):

                    # Append every word in the line into the word array.
                    for word in currentLine.split(' '):
                        
                        # Split the word and tag.
                        try:
                            tokenTuple = word.split('/')
                            wordArray.append((tokenTuple[0], tokenTuple[1]))
                        except:
                            pass

                    # print(wordArray)
                    sentenceArray.append(wordArray)

        # Exit otherwise.
        except IOError:
            sys.exit("Cannot read file")

    return sentenceArray

# test the function here:
path = "/content/train/"
data = load_corpus(path)
print(data[0])
print(len(data))
print(len(data[0]))

[('Local', 'ADJECTIVE'), ("industry's", 'NOUN'), ('investment', 'NOUN'), ('in', 'PREPOSITION'), ('Rhode', 'NOUN'), ('Island', 'NOUN'), ('was', 'VERB'), ('the', 'DETERMINER'), ('big', 'ADJECTIVE'), ('story', 'NOUN'), ('in', 'PREPOSITION'), ("1960's", 'NUMBER'), ('industrial', 'ADJECTIVE'), ('development', 'NOUN'), ('effort', 'NOUN'), ('.', 'PUNCT')]
57340
16


In [7]:
import numpy as np # convert lists to np arrays before returning them

# Creates the dataset with train_X (words) and train_y (tag).
def create_dataset(sentences):
    
    # Defines the relevant lists.
    train_X, train_y = list(), list()

    # Dictionaries that will provide word/tag to integer mapping
    word2idx, tag2idx = dict(), dict() 

    # Dictionary to convert ids back to the respected words.
    id2wordX = {}
    uniqueWordCount = 0

    # Include a [PAD] token for the lists that aren't the MAX_LENGTH in size. 
    word2idx["[PAD]"] = uniqueWordCount
    id2wordX[uniqueWordCount] = "[PAD]"
    uniqueWordCount += 1

    # Include a [OOV] token for words that can't be found in the dictionary.
    word2idx["[OOV]"] = uniqueWordCount
    id2wordX[uniqueWordCount] = "[OOV]"
    uniqueWordCount += 1

    # Go through each sentence in the data.
    for s in data:
        # For each word.
        for w in s:

            # Convert to lowercase.
            word = w[0].lower()

            # Add it to the word2idx and id2wordx dictionaries.
            if word not in word2idx:
                word2idx[word] = uniqueWordCount
                id2wordX[uniqueWordCount] = word
                uniqueWordCount += 1
    
    # The tag possibilities list.
    tagString = "NOUN, PRONOUN, VERB, ADVERB, ADJECTIVE, CONJUNCTION, PREPOSITION, DETERMINER, NUMBER, PUNCT, X"
    tagArray = tagString.split(', ')

    # Dictionary to convert ids back to the respected tags.
    id2tagX = {}
    uniqueTagCount = 0

    # Include a [PAD] token for the lists that aren't the MAX_LENGTH in size. 
    tag2idx["[PAD]"] = uniqueTagCount
    id2tagX[uniqueTagCount] = "[PAD]"
    uniqueTagCount += 1

    # For each tag in the tag list.
    for tag in tagArray:

        # Add it to the tag2idx and id2tagx dictionaries if it doesn't exist.
        if tag not in tag2idx:
            tag2idx[tag] = uniqueTagCount
            id2tagX[uniqueTagCount] = tag
            uniqueTagCount += 1

    # Deep copy the sentences array for train_X and train_y.
    train_X = [row[:] for row in sentences]
    train_y = [row[:] for row in sentences]

    # For each sentence in the sentences list.
    for s in range(len(sentences)):
        
        # For each word in the sentence.
        for w in range(len(sentences[s])):

            # Get the (lowercase) word and tag, add it to train_X and train_y.
            word = sentences[s][w][0].lower()
            tag = sentences[s][w][1]
            train_X[s][w] = word2idx[word]
            train_y[s][w] = tag2idx[tag]

    # Convert into a numpy array.
    train_X = np.array(train_X)
    train_y = np.array(train_y)

    # Output train_X, train_y, and the relevant dictionries.
    return train_X, train_y, word2idx, tag2idx, id2wordX, id2tagX 

# Test the function
train_X, train_y, word2idx, tag2idx, id2wordX, id2tagX = create_dataset(data)
print(train_X[0])
print(train_y[0])
print(id2wordX[train_X[0][0]])
print(id2tagX[train_y[0][0]])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 5, 12, 13, 14, 15, 16]
[5, 1, 1, 7, 1, 1, 3, 8, 5, 1, 7, 9, 5, 1, 1, 10]
local
ADJECTIVE


In [8]:
from keras.preprocessing.sequence import pad_sequences as pad

# Pad the sequences with 0s to the max length.
def pad_sequences(train_X, train_y):

    # Used to find the max array size.
    tempMax = -1

    # Keep and store the max length of a sentence as it goes through train_X.
    for s in train_X:
        if (len(s) > tempMax):
            tempMax = len(s)

    # Define the MAX_LENGTH of a sentence.
    MAX_LENGTH = tempMax

    # Pad train_X and train_y.
    train_X = pad(train_X, MAX_LENGTH, 'int32', 'post', 'pre', 0.0)
    train_y = pad(train_y, MAX_LENGTH, 'int32', 'post', 'pre', 0.0)
    
    # Return the numpy arrays and MAX-LENGTH.
    return train_X, train_y, MAX_LENGTH

# Test the function
train_X, train_y, MAX_LENGTH = pad_sequences(train_X, train_y)
print(train_X[0])
print(train_y[0])
print(MAX_LENGTH)

[ 2  3  4  5  6  7  8  9 10 11  5 12 13 14 15 16  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
[ 5  1  1  7  1  1  3  8  5  1  7  9  5  1  1 10  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0 

In [9]:
from keras.models import Sequential
from keras.layers import InputLayer, Activation
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from keras.optimizers import Adam

# Define the Keras model.
def define_model(MAX_LENGTH):  
    model = Sequential()
    model.add(InputLayer(input_shape=(MAX_LENGTH, ))) # MAX_LENGTH is the max length of each sequence, as output by previous method

    # Adds an Embedding layer, Bidirectional layer, and is a Dense netowrk.
    model.add(Embedding(len(word2idx), 128))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(len(tag2idx))))

    # Add the softmax activation and the Adam optimizer for loss.
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
  
    # Print the model summary.
    print (model.summary())
    return model

# Call the function here
model = define_model(MAX_LENGTH)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 180, 128)          6367104   
_________________________________________________________________
bidirectional (Bidirectional (None, 180, 512)          788480    
_________________________________________________________________
time_distributed (TimeDistri (None, 180, 12)           6156      
_________________________________________________________________
activation (Activation)      (None, 180, 12)           0         
Total params: 7,161,740
Trainable params: 7,161,740
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# Returns the one-hot encoding of the sequence.
def to_categorical(sequences, categories = 11):

    # Creates the sentence list.
    sentenceList = []

    # For each sentence in the seqeuences list.
    for s in range(len(sequences)):

        # Create a word list for each sentence.
        wordList = []

        # For each word in the sentence.
        for w in range(len(sequences[s])):

            # Get the integer value of the word.
            tempValue = sequences[s][w]

            # Create list with with 0s the length of the categories.
            tempList = np.zeros(categories)

            # Fill the 1 in the relevant 1-hot position and append it to
            # the word List.
            tempList[tempValue] = 1
            wordList.append(tempList)

        # Append the wordList to the sentence.
        sentenceList.append(wordList)

    # Convert sentenceList to a Numpy array.
    sentenceList = np.asarray(sentenceList)

    # Return it.
    return np.asarray(sentenceList)

# Call the function here
train_y1 = to_categorical(train_y, categories = len(tag2idx))
print(train_y[0])
print(train_y1[0][0])
print(train_y1[0][1])

[ 5  1  1  7  1  1  3  8  5  1  7  9  5  1  1 10  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [11]:
import tensorflow as tf

# Trains the model.
def train(model, train_X, train_y1):

    # Fit the data into the Keras model, through 40 passes (epochs).
    model.fit(train_X, train_y1, batch_size=128, epochs=40, validation_split=0.2)

    # Return the model.
    return model

model = train(model, train_X, train_y1)

Epoch 1/40
359/359 [==============================] - 42s 117ms/step - loss: 0.2035 - accuracy: 0.9408 - val_loss: 0.0374 - val_accuracy: 0.9905
Epoch 2/40
359/359 [==============================] - 41s 115ms/step - loss: 0.0185 - accuracy: 0.9950 - val_loss: 0.0152 - val_accuracy: 0.9953
Epoch 3/40
359/359 [==============================] - 42s 116ms/step - loss: 0.0092 - accuracy: 0.9971 - val_loss: 0.0130 - val_accuracy: 0.9958
Epoch 4/40
359/359 [==============================] - 41s 116ms/step - loss: 0.0069 - accuracy: 0.9978 - val_loss: 0.0123 - val_accuracy: 0.9961
Epoch 5/40
359/359 [==============================] - 42s 116ms/step - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.0120 - val_accuracy: 0.9962
Epoch 6/40
359/359 [==============================] - 41s 115ms/step - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.0123 - val_accuracy: 0.9962
Epoch 7/40
359/359 [==============================] - 42s 116ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0127 - val_ac

In [16]:
# Convert the sentence to one the Keras model can use (integer as words).
def convertModelInput(sentence, MAX_LENGTH):

    # Create a sentence array.
    sentArray = []

    # If the sentence is a string convert it into an array.
    if type(sentence) == str:
        sentence = sentence.lower()
        sentArray = sentence.split(' ')
        print(sentArray)

    # Otherwise, just copy it directly to the sentArray.
    elif type(sentence) == list:
        sentArray = sentence.copy()
        print(sentArray)
    
    else:
        sys.exit("Incorrect input to method")

    # Creates a test_X list.
    test_X = []

    # For each word in the test setence.
    for word in sentArray:

        # Try finding the word to integer value in the dictionary & append it.
        try:
            test_X.append(word2idx[word.lower()])

        # Otherwise, use an OOV identifier and append it to test_X.
        except:
            test_X.append(word2idx["[OOV]"])

    # Append a 0 to test_X until it the sentence is MAX_LENGTH.
    for i in range(MAX_LENGTH - len(sentArray)):
        test_X.append(0)

    # Return test_X as an Numpy array.
    return np.array(test_X)

# Test the sentence in the given model.
def test(model, sentence):

    # Convert the sentence to an integer Numpy array the model can use.
    test_X = convertModelInput(sentence, MAX_LENGTH)

    # Get the prediction tag results (as an integer list) from the model.
    intResults = model.predict_classes(test_X)

    # Create an array for the tag (in English).
    tagResults = []
    
    # For each sentence in the results.
    for sentence in intResults:

        # For each tag in the setence, append the relevand tag from the integer.
        for tag in sentence:
            tagResults.append(id2tagX[tag])
    
    # Return the tag list.
    return tagResults

# For the first evaluation sentence.
testString1 = ["the", "secretariat", "is", "expected", "to", "race", "tomorrow", "."]
tag1 = test(model, testString1)
print(tag1)
print("")

# For the second evaluation sentence.
testString2 = "people continue to enquire the reason for the race for outer space ."
tag2 = test(model, testString2)
print(tag2)


['the', 'secretariat', 'is', 'expected', 'to', 'race', 'tomorrow', '.']
['DETERMINER', 'PRONOUN', 'VERB', 'VERB', 'PREPOSITION', 'NOUN', 'NOUN', 'PUNCT', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [13]:
# Delete the /train/ folder 
# (Google colab does not the deletion of a folder if it still has contents).
!rm -dr /content/train/